In [172]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import warnings
warnings.filterwarnings('ignore')

In [173]:
# #텍스트 정제(전처리)
# def cleanText(readData):
#     #스팸 메세지에 포함되어 있는 특수 문자 제거
#     text = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '', readData)
#     #양쪽(위,아래)줄바꿈 제거/
#     text = text.strip('\n')
#     return text

##### 하나에 대해 해보기

In [174]:
# 알파벳 소문자로 변환
train_ex = open('./lang/train/en-1.txt', 'r', encoding='utf-8')
train_str_ex = train_ex.read().lower()
train_ex.close()

test_ex = open('./lang/test/en-1.txt', 'r', encoding='utf-8')
test_str_ex = test_ex.read().lower()
test_ex.close()

# 알파벳 빈도수 계산
Alphabet='abcdefghijklmnopqrstuvwxyz'
train_freq_ex = [0]*26 # train 알파벳 26자
test_freq_ex = [0]*26 # test 알파벳 26자

for c in train_str_ex:
    if c in Alphabet:
        idx = Alphabet.find(c)
        train_freq_ex[idx] += 1

for c in test_str_ex:
    if c in Alphabet:
        idx = Alphabet.find(c)
        test_freq_ex[idx] += 1

# 알파벳 빈도수 출력
# print('train data에 대한 알파벳 빈도수')
# for i in range(len(Alphabet)):
#     print(f'{Alphabet[i]} : {train_freq_ex[i]}')

# print('test data에 대한 알파벳 빈도수')
# for i in range(len(Alphabet)):
#     print(f'{Alphabet[i]} : {test_freq_ex[i]}')

# a ~ z 컬럼명 담기
Alphabet_names = []
for i in range(len(Alphabet)):
    Alphabet_names.append(Alphabet[i])

# DataFrame으로 만들기
# pd.DataFrame(zip(Alphabet_names, train_freq_ex), columns=['Alphabet', 'freq'])

# 딕셔너리로 만들기
# dict(zip(Alphabet_names, train_freq_ex))

### 전체에 대해

In [175]:
df=pd.DataFrame(columns = Alphabet_names)
df

,a,b,c,d,e,f,g,h,i,j,...,q,r,s,t,u,v,w,x,y,z


In [176]:
# train data
train_listdir = os.listdir('./lang/train/') # 해당 경로의 폴더 내 모든 파일 리스트로
Alphabet='abcdefghijklmnopqrstuvwxyz'
for l in train_listdir:
    train_freq = [0]*26
    train = open(f'./lang/train/{l}', 'r', encoding='utf-8')
    train_str = train.read().lower() # 알파벳 소문자로
    for c in train_str:
        if c in Alphabet:
            idx = Alphabet.find(c) # 알파벳 분류 및 담기
            train_freq[idx] += 1

    for j in range(len(train_freq)):
        train_freq[j] = train_freq[j] / sum(train_freq) # 비율로 변환
    df = df.append(pd.Series(train_freq, index=Alphabet_names), ignore_index=True) # 데이터프레임으로 변환, 담을 때 리스트에서 시리즈로 변환
    train.close()

In [177]:
df['target'] = ['en', 'en','en','en','en','fr','fr','fr','fr','fr','id','id','id','id','id','tl','tl','tl','tl','tl']
df.head()
# 타겟데이터 설정, 국가별로

,a,b,c,d,e,f,g,h,i,j,...,r,s,t,u,v,w,x,y,z,target
0,0.075952,0.013895,0.050154,0.053305,0.128546,0.021942,0.027420,0.064395,0.116421,0.003100,...,0.267184,0.287924,0.530304,0.362536,0.214689,0.394369,0.029936,0.946070,0.323078,en
1,0.084178,0.021742,0.033936,0.044909,0.165725,0.025241,0.046467,0.042779,0.122803,0.004873,...,0.289533,0.323920,0.519639,0.426034,0.331733,0.504319,0.147186,0.912341,0.523032,en
2,0.071646,0.013111,0.049818,0.037494,0.143273,0.020421,0.035794,0.034674,0.144523,0.004445,...,0.178551,0.354395,0.505471,0.365646,0.372544,0.374235,0.027672,0.924569,0.321151,en
3,0.072210,0.029872,0.033305,0.045502,0.145737,0.023649,0.034020,0.088184,0.106907,0.002768,...,0.196849,0.303591,0.556780,0.325602,0.100929,0.432225,0.233496,0.883350,0.683666,en
4,0.073806,0.021991,0.034332,0.045317,0.169150,0.029353,0.030240,0.087178,0.109091,0.006184,...,0.236916,0.255074,0.548696,0.317407,0.199611,0.554208,0.158674,0.914104,0.410740,en


In [178]:
df['target'].count()

20

### 분석

In [179]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate
from sklearn.utils import all_estimators

In [180]:
train_X, test_X, train_y, test_y = train_test_split(df[df.columns[0:-1]], df['target'], test_size=0.2, random_state=42)
                                                         # input data    # input target

In [181]:
# 적합한 모델 찾기
scores_list=[]
models = all_estimators(type_filter='classifier')

for name, model in models:
    try:
        # 모델 객체를 생성
        md = model()
        # 학습
        md.fit(train_X, train_y)
        # 평가
        model_score = md.score(test_X, test_y)
        scores_list.append((name, np.round(model_score, 4)))
    except:
        pass

scores_list

[('AdaBoostClassifier', 1.0),
 ('BaggingClassifier', 1.0),
 ('BernoulliNB', 0.0),
 ('CalibratedClassifierCV', 0.75),
 ('CategoricalNB', 0.0),
 ('ComplementNB', 0.25),
 ('DecisionTreeClassifier', 0.5),
 ('DummyClassifier', 0.0),
 ('ExtraTreeClassifier', 0.25),
 ('ExtraTreesClassifier', 1.0),
 ('GaussianNB', 1.0),
 ('GaussianProcessClassifier', 0.0),
 ('GradientBoostingClassifier', 1.0),
 ('HistGradientBoostingClassifier', 0.0),
 ('KNeighborsClassifier', 0.75),
 ('LabelPropagation', 1.0),
 ('LabelSpreading', 1.0),
 ('LinearDiscriminantAnalysis', 0.75),
 ('LinearSVC', 0.75),
 ('LogisticRegression', 0.0),
 ('LogisticRegressionCV', 0.75),
 ('MLPClassifier', 1.0),
 ('MultinomialNB', 0.0),
 ('NearestCentroid', 0.75),
 ('NuSVC', 1.0),
 ('PassiveAggressiveClassifier', 0.75),
 ('Perceptron', 0.75),
 ('QuadraticDiscriminantAnalysis', 0.5),
 ('RadiusNeighborsClassifier', 0.0),
 ('RandomForestClassifier', 1.0),
 ('RidgeClassifier', 0.75),
 ('RidgeClassifierCV', 0.75),
 ('SGDClassifier', 0.75),
 ('S

In [182]:
# np.array 또는 train_X.values 사용
train_X = np.array(train_X)
train_y = np.array(train_y)

# 모델 생성
forest_model = RandomForestClassifier(max_depth=3, oob_score=True)
forest_model.fit(train_X, train_y)
# 모델 평가
print('train score : ', forest_model.score(train_X, train_y))
print('test score : ', forest_model.score(test_X, test_y))
print('oob_score_ : ', forest_model.oob_score_)

train score :  1.0
test score :  1.0
oob_score_ :  0.875


In [183]:
# 교차 검증 후 평가점수 확인
result = cross_validate(forest_model, df[df.columns[0:-1]], df['target'], return_train_score=True, cv=3)
print(result)
print(result['test_score'].mean())

{'fit_time': array([0.09865689, 0.08630157, 0.08968902]), 'score_time': array([0.00797296, 0.00797176, 0.00598001]), 'test_score': array([1., 1., 1.]), 'train_score': array([1., 1., 1.])}
1.0


In [184]:
# 실제로 테스트할 모델 지정

def test_lang():
    user_text = input('확인할 언어를 입력하십시오 : ')
    language_lower = user_text.lower()
    
    Alphabet = 'abcdefghijklmnopqrstuvwxyz'
    Alphabet_freq = [0]*26
    for al in language_lower:
        if al in Alphabet:
            idx = Alphabet.find(al)
            Alphabet_freq[idx]+=1

    for j in range(len(Alphabet_freq)):
        Alphabet_freq[j] = Alphabet_freq[j] / sum(Alphabet_freq)
    print('입력한 언어에 해당하는 언어는 ', forest_model.predict([Alphabet_freq])[0])



In [185]:
test_lang()

입력한 언어에 해당하는 언어는  en


In [187]:
# 테스트 해본 문장 : 
# Microsoft Windows (or simply Windows) is a metafamily of graphical operating systems developed, marketed, and sold by Microsoft. It consists of several families of operating systems, each of which cater to a certain sector of the computing industry. Active Windows families include Windows NT, Windows Embedded and Windows Phone; these may encompass subfamilies, e.g. Windows Embedded Compact (Windows CE) or Windows Server. Defunct Windows families include Windows 9x; Windows 10 Mobile is an active product, unrelated to the defunct family Windows Mobile.
# Microsoft introduced an operating environment named Windows on November 20, 1985, as a graphical operating system shell for MS-DOS in response to the growing interest in graphical user interfaces (GUIs).[4] Microsoft Windows came to dominate the world's personal computer (PC) market with over 90% market share, overtaking Mac OS, which had been introduced in 1984. On PCs, Windows is still the most popular operating system. However, in 2014, Microsoft admitted losing the majority of the overall operating system market to Android,[5] because of the massive growth in sales of Android smartphones. In 2014, the number of Windows devices sold were less than 25% of Android devices sold. This comparisons, however, may not be fully relevant as the two operating systems traditionally targeted different platforms.
# As of March 2016[update], the most recent version of Windows for PCs, tablets, smartphones and embedded devices is Windows 10. The most recent versions for server computers is Windows Server 2012 R2. A specialized version of Windows runs on the Xbox One game console.[6]